In [13]:
# sourcery skip: for-index-underscore
import contextlib
import pandas as pd
import requests
from datetime import *
from bs4 import BeautifulSoup

In [14]:
url = "https://www.tmz.com"
req = requests.get(url)
doc = BeautifulSoup(req.text)

In [15]:
# init var headline
tmz = doc.select("header a h2")

In [16]:
# init var timestamp
tmz_timestamps = doc.select(".article")

In [17]:
# main loop
stories = pd.DataFrame()
paragraphs = []
count = 0

for trash in enumerate(tmz):
    # grab headline text
    headline = tmz[count].text
    headline = headline.replace("\n", " ")

    # grab, clean timestamp
    timestamp = tmz_timestamps[count].text.split("PT")[-20:]
    timestamp = timestamp[0][-20:]
    timestamp = timestamp.strip()

    # set timestamp, date, time, headlie
    story = {
        "timestamp": timestamp,
        "date": timestamp[:9],
        "time": timestamp[-8:].strip(),
        "headline": headline,
    }

    # convert to df
    story = pd.DataFrame([story])

    # convert story 12AP to 24
    try:
        story["time"] = pd.to_datetime(
            story["time"], format="%I:%M %p"
        ).dt.strftime("%H:%M")
    except:
        story["time"] = story["time"]
        
    story.convert_dtypes()
    
    story['year'] = pd.DatetimeIndex(story['date']).year
    story['month'] = pd.DatetimeIndex(story['date']).month
    story['day'] = pd.DatetimeIndex(story['date']).day
    story['hour'] = pd.DatetimeIndex(story['time']).hour
    story['minute'] = pd.DatetimeIndex(story['time']).minute
    # parse date into columns
    # try:
    #     story["year"] = story['date'].astype(str)[-4:]
    # except:
    #     story["year"] = story["year"]

    # append story to raw stories df to be merged with historical df
    stories = pd.concat([stories, story], ignore_index=True)

    count += 1

stories = stories[["headline", "timestamp", "year", "month", "day", "hour", "minute"]]
stories

,headline,timestamp,year,month,day,hour,minute
0,"Britney Spears Will Not Be Deposed, Judge Rules",7/27/2022 3:26 PM,2022,7,27,15,26
1,Mike Tyson Using Cane To Get Around ... But I...,7/27/2022 3:56 PM,2022,7,27,15,56
2,Jeff Bezos Parents Buy $34M Mega Mansion in FL,7/27/2022 3:01 PM,2022,7,27,15,1
3,Brittney Griner I Flew To Russia Despite Trav...,7/27/2022 1:06 PM,2022,7,27,13,6
4,Tony Dow 'Leave It To Beaver' Star Dead At 77,7/27/2022 12:27 PM,2022,7,27,12,27
5,Joey Chestnut Downs 44 Cane's Chicken Fingers...,7/27/2022 2:06 PM,2022,7,27,14,6
6,Beyoncé ‘Renaissance’ Album Leak Beyhive Says...,7/27/2022 12:02 PM,2022,7,27,12,2
7,Ukrainian Prez Volodymyr Zelensky Dragged Ove...,7/27/2022 10:15 AM,2022,7,27,10,15
8,GEORGE FLOYD'S DEATH Former Cops Get Prison T...,7/27/2022 10:14 AM,2022,7,27,10,14
9,Shawn Mendes Cancels Tour ... For Mental Heal...,7/27/2022 7:17 AM,2022,7,27,7,17


In [18]:
existing = pd.read_csv("../headlines.csv")
existing = existing.dropna(axis=1).set_index("Unnamed: 0")
existing.convert_dtypes()

existing["year"] = pd.DatetimeIndex(existing["timestamp"]).year
existing["month"] = pd.DatetimeIndex(existing["timestamp"]).month
existing["day"] = pd.DatetimeIndex(existing["timestamp"]).day
existing["hour"] = pd.DatetimeIndex(existing["timestamp"]).hour
existing["minute"] = pd.DatetimeIndex(existing["timestamp"]).minute

existing

,headline,timestamp,year,month,day,hour,minute
Unnamed: 0,,,,,,,
158,Jamie Foxx Wild Sing-along on Italian Vacay ....,7/27/2022 8:02 AM,2022,7,27,8,2
157,Jerry Jones Apologizes For Derogatory Term .....,7/26/2022 4:23 PM,2022,7,26,16,23
155,'Top Chef' Star Howard Kleinberg Dead at 46,7/27/2022 7:39 AM,2022,7,27,7,39
156,Norman Lear Singin' 'That's Amore' At 100 ......,7/27/2022 8:14 AM,2022,7,27,8,14
0,Mike Tyson Using Cane To Get Around ... But I...,7/27/2022 3:56 PM,2022,7,27,15,56
...,...,...,...,...,...,...,...
150,Ukrainian Prez Volodymyr Zelensky Dragged Ove...,7/27/2022 10:15 AM,2022,7,27,10,15
151,GEORGE FLOYD'S DEATH Former Cops Get Prison T...,7/27/2022 10:14 AM,2022,7,27,10,14
152,Shawn Mendes Cancels Tour ... For Mental Heal...,7/27/2022 7:17 AM,2022,7,27,7,17


In [32]:
try:
    existing = pd.read_csv("headlines.csv")
    existing.convert_dtypes()
    existing["year"] = pd.DatetimeIndex(existing["timestamp"]).year
    existing["month"] = pd.DatetimeIndex(existing["timestamp"]).month
    existing["day"] = pd.DatetimeIndex(existing["timestamp"]).day
    existing["hour"] = pd.DatetimeIndex(existing["timestamp"]).hour
    existing["minute"] = pd.DatetimeIndex(existing["timestamp"]).minute
    existing = existing[
        ["headline", "timestamp", "date", "time", "year", "month", "day", "hour", "minute"]
    ]
except:
    pass
    
export = pd.concat([existing, stories], ignore_index=True)
export = export[
    ["headline", "timestamp", "year", "month", "day", "hour", "minute"]
]

export = export.drop_duplicates(subset=['timestamp'], keep="first")
export = export.sort_values(by=["year", "month", "day", "hour", "minute"], ascending=[False, False, False, False, False], ignore_index=True)
export

,headline,timestamp,year,month,day,hour,minute
0,Mike Tyson Using Cane To Get Around ... But I...,7/27/2022 3:56 PM,2022,7,27,15,56
1,"Britney Spears Will Not Be Deposed, Judge Rules",7/27/2022 3:26 PM,2022,7,27,15,26
2,Jeff Bezos Parents Buy $34M Mega Mansion in FL,7/27/2022 3:01 PM,2022,7,27,15,1
3,Joey Chestnut Downs 44 Cane's Chicken Fingers...,7/27/2022 2:06 PM,2022,7,27,14,6
4,Brittney Griner I Flew To Russia Despite Trav...,7/27/2022 1:06 PM,2022,7,27,13,6
5,Tony Dow 'Leave It To Beaver' Star Dead At 77,7/27/2022 12:27 PM,2022,7,27,12,27
6,Beyoncé ‘Renaissance’ Album Leak Beyhive Says...,7/27/2022 12:02 PM,2022,7,27,12,2
7,Ukrainian Prez Volodymyr Zelensky Dragged Ove...,7/27/2022 10:15 AM,2022,7,27,10,15
8,GEORGE FLOYD'S DEATH Former Cops Get Prison T...,7/27/2022 10:14 AM,2022,7,27,10,14
9,Kanye West Gifts A$AP Bari New Maybach… Days ...,7/27/2022 8:18 AM,2022,7,27,8,18


In [33]:
export.to_csv("headlines.csv")